In [1]:
%load_ext advent_of_code

In [2]:
import re
import math
import itertools

from collections import defaultdict, Counter

from tqdm.auto import tqdm

import numpy as np
from functools import cache

# Day 1

In [3]:
%%aoc 1 1

answer = 0
for line in inputs.strip().split("\n"):
    num = [c for c in line if c.isdigit()]
    answer += int(num[0] + num[-1])

Execution time: 0.0065 seconds
Correct!


In [4]:
%%aoc 1 2

digits = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
digit_map = {d: i for i, d in enumerate(digits, 1)}

answer = 0
for line in inputs.strip().split("\n"):
    num = re.findall("(?=(\d|"+"|".join(digits)+"))", line)
    num = [digit_map.get(i, None) or int(i) for i in num]
    answer += num[0] * 10 + num[-1]

Execution time: 0.014 seconds
Correct!


# Day 2

In [5]:
%%aoc 2 1

total = {"red": 12, "green": 13, "blue": 14}

answer = 0

for game in inputs.strip().split("\n"):
    num, rounds = game.split(": ", 1)
    num = int(num[5:])

    possible = True
    for round in rounds.split("; "):
        for cubes in round.split(", "):
            count, color = cubes.split()
            if int(count) > total[color]:
                possible = False

    if possible:
        answer += num

Execution time: 0.006 seconds
Correct!


In [6]:
%%aoc 2 2

answer = 0

for game in inputs.strip().split("\n"):
    num, rounds = game.split(": ", 1)
    num = int(num[5:])

    total = {"red": 0, "green": 0, "blue": 0}


    for round in rounds.split("; "):
        for cubes in round.split(", "):
            count, color = cubes.split()
            total[color] = max(total[color], int(count))

    power = 1
    for count in total.values():
        power *= count

    answer += power

Execution time: 0.0094 seconds
Correct!


# Day 3

In [7]:
%%aoc 3 1

grid = inputs.strip().split("\n")

grid = ["." + line + "." for line in grid]
grid = ["."*len(grid[0])] + grid + ["."*len(grid[0])]

answer = 0
for i, line in enumerate(grid[1:-1], 1):
    for num in re.finditer(r"(?<!\d)\d+(?=.)", line):
        start, end = num.span()
        if not (set(grid[i-1][start-1:end+1] + grid[i+1][start-1:end+1] + line[start-1] + line[end]) == {"."}):
            answer += int(num.group())

Execution time: 0.018 seconds
Correct!


In [8]:
%%aoc 3 2

grid = inputs.strip().split("\n")

grid = ["." + line + "." for line in grid]
grid = ["."*len(grid[0])] + grid + ["."*len(grid[0])]

answer = 0

gear_values = defaultdict(list)

for i, line in enumerate(grid[1:-1], 1):
    for num in re.finditer(r"(?<!\d)\d+(?=.)", line):
        start, end = num.span()
        for row_i in range(i-1, i+2):
            row = grid[row_i][start-1:end+1]
            for gear in re.finditer(r"\*", row):
                gear_values[(row_i, start+gear.start())].append(int(num.group()))

for nums in gear_values.values():
    if len(nums) == 2:
        answer += nums[0] * nums[1]

Execution time: 0.02 seconds
Correct!


# Day 4

In [9]:
inputs = """Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
"""

In [10]:
%%aoc 4 1

answer = 0

for card in inputs.strip().split("\n"):
    i, win, nums = re.match(r"Card +(\d+): ([0-9 ]+) \| ([0-9 ]+)", card).groups()
    win = Counter(win.split())
    nums = Counter(nums.split())

    answer += int(2**((win & nums).total()-1))

Execution time: 0.014 seconds
Correct!


In [11]:
%%aoc 4 2

muls = [1 for card in inputs.strip().split("\n")]

for i, card in enumerate(inputs.strip().split("\n")):
    win, nums = re.match(r"Card +\d+: ([0-9 ]+) \| ([0-9 ]+)", card).groups()
    win = Counter(win.split())
    nums = Counter(nums.split())

    for j in range(i+1, i+1+(win & nums).total()):
        muls[j] += muls[i]

answer = sum(muls)

Execution time: 0.013 seconds
Correct!


# Day 5

In [12]:
%%aoc 5 1

seeds, *maps = inputs.strip().split("\n\n")

seeds = [int(seed) for seed in seeds.split()[1:]]
maps = [[[int(x) for x in part.split()] for part in layer.split("\n")[1:]] for layer in maps]

answer = float("inf")
for seed in seeds:
    for layer in maps:
        for dest, source, n in layer:
            if source <= seed < source + n:
                seed = seed - source + dest
                break
    answer = min(answer, seed)

Execution time: 0.0076 seconds
Correct!


In [13]:
%%aoc 5 2

seeds, *maps = inputs.strip().split("\n\n")

seeds = [int(seed) for seed in seeds.split()[1:]]
seeds = [[(s, s+n)] for s, n in zip(seeds[::2], seeds[1::2])]

maps = [[[int(x) for x in part.split()] for part in layer.split("\n")[1:]] for layer in maps]
maps = [[(dest_start, source_start, source_start + layer_n) for dest_start, source_start, layer_n in layer] for layer in maps]

answer = float("inf")
for seed in seeds:
    for layer in maps:
        new_seed = []
        while seed:
            reg_start, reg_end = seed.pop()
            for dest_start, source_start, source_end in layer:
                overlap = (max(reg_start, source_start), min(reg_end, source_end))
                if overlap[0] < overlap[1]:                    
                    new_seed.append((overlap[0] - source_start + dest_start, overlap[1] - source_start + dest_start))
                    if reg_start < source_start:
                        seed.append((reg_start, source_start))
                    if reg_end > source_end:
                        seed.append((source_end, reg_end))
                    break
            else:
                new_seed.append((reg_start, reg_end))
    
        seed = new_seed
    answer = min(answer, *(reg[0] for reg in seed))

Execution time: 0.018 seconds
Correct!


# Day 6

In [14]:
%%aoc 6 1
data = list(zip(*[map(int, line.split()[1:]) for line in inputs.strip().split("\n")]))
answer = 1
for t, d in data:
    answer *= t - 1 - 2*math.floor((t - math.sqrt(t**2 - 4*d))/2)

Execution time: 0.002 seconds
Correct!


In [15]:
%%aoc 6 2
t, d = map(int, ["".join(line.split()[1:]) for line in inputs.strip().split("\n")])
answer = t - 1 - 2*math.floor((t - math.sqrt(t**2 - 4*d))/2)

Execution time: 0.0016 seconds
Correct!


# Day 7

In [16]:
%%aoc 7 1

types = [(5, ), (1, 4), (2, 3), (1, 1, 3), (1, 2, 2), (1, 1, 1, 2), (1, 1, 1, 1, 1)]
cards = ["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3", "2"]

def hand_type(hand):
    hand_type = types.index(tuple(sorted(len(list(x[1])) for x in itertools.groupby(sorted(hand)))))
    first_card = [cards.index(card) for card in hand]
    return (hand_type, first_card)

lines = [line.split(" ") for line in inputs.strip().split("\n")]

hands = sorted(lines, key=lambda x: hand_type(x[0]), reverse=True)

answer = 0
for i, (_, bid ) in enumerate(hands, 1):
    answer += i * int(bid)

Execution time: 0.021 seconds
Correct!


In [17]:
%%aoc 7 2

types = [(5, ), (1, 4), (2, 3), (1, 1, 3), (1, 2, 2), (1, 1, 1, 2), (1, 1, 1, 1, 1)]
cards = ["A", "K", "Q", "T", "9", "8", "7", "6", "5", "4", "3", "2", "J"]

def hand_type(hand):
    counts = tuple(sorted(len(list(x[1])) for x in itertools.groupby(sorted(hand.replace("J", "")))))
    for i, t in enumerate(types):
        if len(counts) <= len(t) and counts <= t:
            hand_type = i
            break
    
    first_card = [cards.index(card) for card in hand]
    return (hand_type, first_card)


lines = [line.split(" ") for line in inputs.strip().split("\n")]

hands = sorted(lines, key=lambda x: hand_type(x[0]), reverse=True)

answer = 0
for i, (_, bid ) in enumerate(hands, 1):
    answer += i * int(bid)

Execution time: 0.029 seconds
Correct!


# Day 8

In [18]:
%%aoc 8 1

path, nodes = inputs.strip().split("\n\n")

path = itertools.cycle(path)

graph = {}
for line in nodes.split("\n"):
    name, adj = line.split(" = ")
    left, right = adj[1:-1].split(", ")
    graph[name] = {"L": left, "R": right}

curr = "AAA"
for answer, turn in enumerate(path, 1):
    curr = graph[curr][turn]
    if curr == "ZZZ":
        break
print(answer)

22411
Execution time: 0.02 seconds
Correct!


In [19]:
%%aoc 8 2

path, nodes = inputs.strip().split("\n\n")

path = itertools.cycle(path)

graph = {}
for line in nodes.split("\n"):
    name, adj = line.split(" = ")
    left, right = adj[1:-1].split(", ")
    graph[name] = {"L": left, "R": right}

currs = [node for node in graph if node.endswith("A")]
lens = [None for _ in currs]

for step, turn in enumerate(path, 1):
    currs = [graph[curr][turn] for curr in currs]
    for i, curr in enumerate(currs):
        if curr.endswith("Z") and lens[i] is None:
            lens[i] = step
    if not any(len is None for len in lens):
        break

answer = math.lcm(*lens)
print(answer)

11188774513823
Execution time: 0.15 seconds
Correct!


# Day 9

In [20]:
%%aoc 9 1

def predict(seq):
    diffs = [y - x for x, y in zip(seq[:-1], seq[1:])]
    if set(diffs) == {0}:
        return seq[-1]

    return seq[-1] + predict(diffs)

seqs = [[int(x) for x in line.split()] for line in inputs.strip().split("\n")]

answer = sum(predict(seq) for seq in seqs)

Execution time: 0.017 seconds
Correct!


In [21]:
%%aoc 9 2

def predict(seq):
    diffs = [y - x for x, y in zip(seq[:-1], seq[1:])]
    if set(diffs) == {0}:
        return seq[0]

    return seq[0] - predict(diffs)

seqs = [[int(x) for x in line.split()] for line in inputs.strip().split("\n")]

answer = sum(predict(seq) for seq in seqs)

Execution time: 0.024 seconds
Correct!


# Day 10

In [22]:
def add(x, y):
    return tuple(np.array(x) + y)

movs = {
    "|": {(0, -1): None, (0, 1): None, (-1, 0): (-1, 0), (1, 0): (1, 0)},
    "-": {(0, -1): (0, -1), (0, 1): (0, 1), (-1, 0): None, (1, 0): None},
    "L": {(0, -1): (-1, 0), (0, 1): None, (-1, 0): None, (1, 0): (0, 1)},
    "J": {(0, -1): None, (0, 1): (-1, 0), (-1, 0): None, (1, 0): (0, -1)},
    "7": {(0, -1): None, (0, 1): (1, 0), (-1, 0): (0, -1), (1, 0): None},
    "F": {(0, -1): (1, 0), (0, 1): None, (-1, 0): (0, 1), (1, 0): None},
    ".": {(0, -1): None, (0, 1): None, (-1, 0): None, (1, 0): None},
    "S": {(0, -1): None, (0, 1): None, (-1, 0): None, (1, 0): None},
}

fills = {
    ((-1, 0), (0, -1)): "J",
    ((-1, 0), (0, 1)): "L",
    ((-1, 0), (1, 0)): "|",
    ((0, -1), (0, 1)): "-",
    ((0, -1), (1, 0)): "7",
    ((0, 1), (1, 0)): "F",
}

In [23]:
%%aoc 10 1

grid = [list("." + row + ".") for row in inputs.strip().split("\n")]
grid = [["."] * len(grid[0])] + grid + [["."] * len(grid[0])]
grid = np.array(grid)

start = list(zip(*np.where(grid == "S")))[0]

poss = {i: (add(start, shift), shift, 1) for i, shift in enumerate(((-1, 0), (1, 0), (0, -1), (0, 1)))}

while len(poss) == len(set(pos for pos, *_ in poss.values())):
    new_poss = {}
    for i, (pos, shift, steps) in poss.items():
        new_shift = movs[grid[pos]][shift]
        if new_shift is None:
            continue
        new_poss[i] = (add(pos, new_shift), new_shift, steps + 1)
    poss = new_poss

answer = poss.popitem()[-1][-1]
print(answer)

6725
Execution time: 0.18 seconds
Correct!


In [24]:
%%aoc 10 2

grid = [list("." + row + ".") for row in inputs.strip().split("\n")]
grid = [["."] * len(grid[0])] + grid + [["."] * len(grid[0])]
grid = np.array(grid)
visited = -np.ones_like(grid, dtype=int)

start = list(zip(*np.where(grid == "S")))[0]

shifts = ((-1, 0), (1, 0), (0, -1), (0, 1))
poss = {i: (add(start, shift), shift, 1) for i, shift in enumerate(shifts)}

while len(poss) == len(set(pos for pos, *_ in poss.values())):
    new_poss = {}
    for i, (pos, shift, steps) in poss.items():
        visited[pos] = i
        new_shift = movs[grid[pos]][shift]
        if new_shift is None:
            continue
        new_poss[i] = (add(pos, new_shift), new_shift, steps + 1)
    poss = new_poss

for i, (pos, *_) in poss.items():
    visited[pos] = i
    visited[start] = i

grid[start] = fills[tuple(sorted(shifts[i] for i in poss))]

answer = 0
for i, row in enumerate(visited):
    inside = False
    for j, cell in enumerate(row):
        if cell in poss:
            if grid[i][j] in {"|", "L", "J"}:
                inside = not inside
        else:
            answer += inside
            if inside:
                visited[i][j] = 5

print(answer)

383
Execution time: 0.22 seconds
Correct!


# Day 11

In [25]:
%%aoc 11 1

grid = np.array([list(row) for row in inputs.strip().split("\n")])
grid = grid == "#"

mul = 2
answer = 0

coords = np.where(grid)[0]

for i in range(len(coords) // 2):
    answer += (mul*(coords[-i-1] - coords[i]) - (mul-1)*(len(set(coords[i:len(coords)-i])) - 2 + 1)) * (len(coords) - 2 * i - 1)

coords = np.where(grid.T)[0]

for i in range(len(coords) // 2):
    answer += (mul*(coords[-i-1] - coords[i]) - (mul-1)*(len(set(coords[i:len(coords)-i])) - 2 + 1)) * (len(coords) - 2 * i - 1)

print(answer)

9550717
Execution time: 0.031 seconds
Correct!


In [26]:
%%aoc 11 2

grid = np.array([list(row) for row in inputs.strip().split("\n")])
grid = grid == "#"

mul = 1000000
answer = 0

coords = np.where(grid)[0]

for i in range(len(coords) // 2):
    answer += (mul*(coords[-i-1] - coords[i]) - (mul-1)*(len(set(coords[i:len(coords)-i])) - 2 + 1)) * (len(coords) - 2 * i - 1)

coords = np.where(grid.T)[0]

for i in range(len(coords) // 2):
    answer += (mul*(coords[-i-1] - coords[i]) - (mul-1)*(len(set(coords[i:len(coords)-i])) - 2 + 1)) * (len(coords) - 2 * i - 1)

print(answer)

648458253817
Execution time: 0.031 seconds
Correct!


# Day 12

In [27]:
@cache
def solve(board, hint):
    if not hint:
        return int("#" not in board)

    total = 0

    max_start = board.find("#")
    if max_start == -1:
        max_start = float("inf")
    max_start = min(max_start, len(board) - sum(hint[1:]) - len(hint) + 1 - hint[0])

    for start in range(max_start + 1):
        if (
            "." not in board[start:start+hint[0]]
            and board[start+hint[0]:start+hint[0]+1] != "#"
        ):
            total += solve(board[start+hint[0]+1:].strip("."), hint[1:])

    return total

In [28]:
%%aoc 12 1

answer = 0

for line in inputs.strip().split("\n"):
    board, hint = line.split()
    hint = tuple(int(x) for x in hint.split(","))

    answer += solve(board, hint)

print(answer)

7173
Execution time: 0.079 seconds
Correct!


In [29]:
%%aoc 12 2

mul = 5

answer = 0

for line in inputs.strip().split("\n"):
    board, hint = line.split()
    hint = tuple(int(x) for x in hint.split(","))

    hint = hint * mul
    board = "?".join([board] * mul)

    answer += solve(board, hint)

print(answer)

29826669191291
Execution time: 0.75 seconds
Correct!


# Day 13

In [30]:
%%aoc 13 1

answer = 0

for grid in inputs.strip().split("\n\n"):
    grid = np.array([[c == "#" for c in row] for row in grid.split("\n")])
    for row_i in range(1, len(grid)):
        forward = grid[row_i:]
        backward = grid[row_i-1::-1]
        size = min(len(forward), len(backward))
        if (forward[:size] == backward[:size]).all():
            answer += (row_i) * 100

    grid = grid.T

    for row_i in range(1, len(grid)):
        forward = grid[row_i:]
        backward = grid[row_i-1::-1]
        size = min(len(forward), len(backward))
        if (forward[:size] == backward[:size]).all():
            answer += (row_i)

print(answer)

30535
Execution time: 0.038 seconds
Correct!


In [31]:
%%aoc 13 2

answer = 0

for grid in inputs.strip().split("\n\n"):
    grid = np.array([[c == "#" for c in row] for row in grid.split("\n")])
    for row_i in range(1, len(grid)):
        forward = grid[row_i:]
        backward = grid[row_i-1::-1]
        size = min(len(forward), len(backward))
        if (forward[:size] != backward[:size]).sum() == 1:
            answer += (row_i) * 100

    grid = grid.T

    for row_i in range(1, len(grid)):
        forward = grid[row_i:]
        backward = grid[row_i-1::-1]
        size = min(len(forward), len(backward))
        if (forward[:size] != backward[:size]).sum() == 1:
            answer += (row_i)

print(answer)

30844
Execution time: 0.04 seconds
Correct!


# Day 14

In [32]:
%%aoc 14 1

def shifter(match):
    counts = Counter(match.group(1))
    return "#" + "O"*counts["O"] + "."*counts["."]

grid = ["#" + "".join(line) + "#" for line in zip(*inputs.strip().split())]
grid = ["#" * len(grid[0])] + grid + ["#" * len(grid[0])]
grid = [re.sub("#([.O]+)", shifter, row) for row in grid]
answer = sum(sum((i * (cell == "O")) for i, cell in enumerate(row[::-1])) for row in grid)
print(answer)

105461
Execution time: 0.013 seconds
Correct!


In [33]:
%%aoc 14 2

def align(grid):
    new_grid = []
    for row in grid:
        new_row = []
        rocks = spaces = 0
        for cell in row:
            if cell == "#":
                new_row.extend(["O"] * rocks + ["."] * spaces + ["#"])
                rocks = spaces = 0
            elif cell == "O":
                rocks += 1
            elif cell == ".":
                spaces += 1
    
        new_grid.append(new_row)
    return new_grid

@cache
def spin(grid):
    for _ in range(4):
        grid = align(grid)
        grid = np.rot90(grid)

    return tuple(tuple(x) for x in grid.tolist())

grid = [("#",) + tuple(line) + ("#", ) for line in zip(*inputs.strip().split())]
grid = tuple([("#",) * len(grid[0])] + grid + [("#", ) * len(grid[0])])

total = 1000000000

seen = {grid: 0}
for i in range(1, total+1):
    grid = spin(grid)
    if grid in seen:
        break
    seen[grid] = i

start = seen[grid]
end = i

for i in range((total-start) % (end-start)):
    grid = spin(grid)

answer = sum(sum((i * (cell == "O")) for i, cell in enumerate(row[::-1])) for row in grid)
print(answer)

102829
Execution time: 3.2 seconds
Correct!


# Day 15

In [34]:
def HASH(text):
    h = 0
    for c in text:
        h = ((h + ord(c)) * 17) & 255
    return h

In [35]:
%%aoc 15 1

answer = sum(HASH(text) for text in inputs.strip().split(","))
print(answer)

504449
Execution time: 0.0054 seconds
Correct!


In [36]:
%%aoc 15 2

boxes = [{} for _ in range(256)]

for instr in inputs.strip().split(","):
    name, command, value = re.match(r"([a-zA-Z]+)(-|=)(\d*)", instr).groups()
    box = HASH(name)
    if command == "-":
        boxes[box].pop(name, None)
    else:
        boxes[box][name] = int(value)

answer = 0
for i, box in enumerate(boxes, 1):
    for j, value in enumerate(box.values(), 1):
        answer += i * j * value

print(answer)

262044
Execution time: 0.019 seconds
Correct!
